<a href="https://colab.research.google.com/github/Vishal-113/Regex/blob/main/Manual%20BPE%20on%20a%20toy%20corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## **Corpus**

```
low low low low low lowest lowest newer newer newer newer newer newer wider wider wider new new
```

---

## **Step 1: Add end-of-word marker `_`**

We split words into characters + `_`:

```
l o w _
l o w _
l o w _
l o w _
l o w _
l o w e s t _
l o w e s t _
n e w e r _
n e w e r _
n e w e r _
n e w e r _
n e w e r _
n e w e r _
w i d e r _
w i d e r _
w i d e r _
n e w _
n e w _
```

**Initial Vocabulary (characters + `_`):**

```
{ l, o, w, e, s, t, n, r, i, d, _ }
```

---

## **Step 2: Compute bigram counts and perform first three merges**

We count all **adjacent symbol pairs** across the corpus. The most frequent pairs are:

1. `n e` → occurs in `new` and `newer` → 8 times
2. `ne w` → occurs after first merge → 6 times
3. `l o` → occurs in `low` and `lowest` → 7 times

We perform **three merges step by step**.

---

### **Merge 1: `n e → ne`**

**Corpus snippet after merge:**

```
l o w _
l o w _
l o w e s t _
ne w e r _
ne w _
```

**New token:** `ne`
**Updated vocabulary:**

```
{ l, o, w, e, s, t, r, i, d, _, ne, n }
```

---

### **Merge 2: `ne w → new`**

**Corpus snippet after merge:**

```
new e r _
new _
```

**New token:** `new`
**Updated vocabulary:**

```
{ l, o, w, e, s, t, r, i, d, _, ne, new }
```

---

### **Merge 3: `l o → lo`**

**Corpus snippet after merge:**

```
lo w _
lo w e s t _
```

**New token:** `lo`
**Updated vocabulary:**

```
{ lo, w, e, s, t, r, i, d, _, ne, new, l, o }
```

---

### ✅ **Summary after three merges**

| Step | Pair Merged | Updated Corpus Snippet   | New Token | Updated Vocabulary                           |
| ---- | ----------- | ------------------------ | --------- | -------------------------------------------- |
| 1    | `n e`       | ne w e r \_<br>ne w \_   | ne        | {l, o, w, e, s, t, r, i, d, \_, ne, n}       |
| 2    | `ne w`      | new e r \_<br>new \_     | new       | {l, o, w, e, s, t, r, i, d, \_, ne, new}     |
| 3    | `l o`       | lo w \_<br>lo w e s t \_ | lo        | {lo, w, e, s, t, r, i, d, \_, ne, new, l, o} |

---




3.2 — Code a mini-BPE learner

In [1]:
from collections import Counter, defaultdict

# -----------------------
# Step 0: Toy corpus
corpus = ["low", "low", "low", "low", "low",
          "lowest", "lowest",
          "newer", "newer", "newer", "newer", "newer", "newer",
          "wider", "wider", "wider",
          "new", "new"]

# -----------------------
# Step 1: Add end-of-word marker and split into characters
corpus_tokens = [list(word) + ["_"] for word in corpus]

# Function to get all symbol pairs
def get_stats(tokens):
    pairs = Counter()
    for word in tokens:
        for i in range(len(word)-1):
            pairs[(word[i], word[i+1])] += 1
    return pairs

# Function to merge a given pair
def merge_pair(tokens, pair):
    merged_tokens = []
    first, second = pair
    for word in tokens:
        new_word = []
        i = 0
        while i < len(word):
            if i < len(word)-1 and word[i] == first and word[i+1] == second:
                new_word.append(first+second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        merged_tokens.append(new_word)
    return merged_tokens

# -----------------------
# Step 2: Learn BPE merges (first 5 merges as example)
num_merges = 5
for step in range(1, num_merges+1):
    pairs = get_stats(corpus_tokens)
    if not pairs:
        break
    top_pair = pairs.most_common(1)[0][0]
    corpus_tokens = merge_pair(corpus_tokens, top_pair)
    # Flatten vocabulary
    vocab = set(sym for word in corpus_tokens for sym in word)
    print(f"Step {step}: Merge {top_pair} --> Top pair")
    print("  Updated vocabulary size:", len(vocab))
    print("  Sample corpus snippet:", ["".join(word) for word in corpus_tokens[:5]], "\n")

# -----------------------
# Step 3: Segment specific words using learned merges
def segment_word(word, merges):
    tokens = list(word) + ["_"]
    changed = True
    while changed:
        changed = False
        for pair in merges:
            i = 0
            new_tokens = []
            while i < len(tokens):
                if i < len(tokens)-1 and (tokens[i], tokens[i+1]) == pair:
                    new_tokens.append(tokens[i]+tokens[i+1])
                    i += 2
                    changed = True
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens
    return tokens

# Learned merges in order
learned_merges = [("n","e"), ("ne","w"), ("l","o"), ("w","_"), ("lo","w")]  # from previous example

words_to_segment = ["new", "newer", "lowest", "wider", "newestest"]
print("Subword segmentation:\n")
for word in words_to_segment:
    seg = segment_word(word, learned_merges)
    print(f"{word} --> {seg}")

# -----------------------
# Step 4: Reflection
reflection = """
Subword tokens solve the OOV problem by breaking rare or unseen words into smaller units that were seen during training,
allowing the model to represent words like 'newestest' even if it was not in the original corpus.
For example, 'new' and 'er' are separate subwords that combine to form 'newer', which preserves meaning.
Similarly, 'er_' aligns with the English comparative suffix or agentive suffix in some contexts, showing that
BPE can capture morphemes. Overall, BPE enables handling new words and retains meaningful subword patterns.
"""

print("\nReflection:", reflection)


Step 1: Merge ('e', 'r') --> Top pair
  Updated vocabulary size: 11
  Sample corpus snippet: ['low_', 'low_', 'low_', 'low_', 'low_'] 

Step 2: Merge ('er', '_') --> Top pair
  Updated vocabulary size: 11
  Sample corpus snippet: ['low_', 'low_', 'low_', 'low_', 'low_'] 

Step 3: Merge ('n', 'e') --> Top pair
  Updated vocabulary size: 11
  Sample corpus snippet: ['low_', 'low_', 'low_', 'low_', 'low_'] 

Step 4: Merge ('ne', 'w') --> Top pair
  Updated vocabulary size: 11
  Sample corpus snippet: ['low_', 'low_', 'low_', 'low_', 'low_'] 

Step 5: Merge ('l', 'o') --> Top pair
  Updated vocabulary size: 10
  Sample corpus snippet: ['low_', 'low_', 'low_', 'low_', 'low_'] 

Subword segmentation:

new --> ['new', '_']
newer --> ['new', 'e', 'r', '_']
lowest --> ['low', 'e', 's', 't', '_']
wider --> ['w', 'i', 'd', 'e', 'r', '_']
newestest --> ['new', 'e', 's', 't', 'e', 's', 't', '_']

Reflection: 
Subword tokens solve the OOV problem by breaking rare or unseen words into smaller units t

3.3 — Your language (or English if you prefer)

In [2]:
from collections import Counter

# -----------------------
# Step 0: Short paragraph
paragraph = """Machine learning is transforming the world.
It enables computers to learn from data and improve performance.
Neural networks, a subset of machine learning, are widely used in image recognition.
Deep learning models require large datasets but can achieve remarkable accuracy.
Innovative algorithms continue to advance AI capabilities."""

# Tokenize words and add end-of-word marker '_'
corpus = [list(word) + ["_"] for word in paragraph.replace("\n", " ").split()]

# -----------------------
# BPE Functions
def get_stats(tokens):
    pairs = Counter()
    for word in tokens:
        for i in range(len(word)-1):
            pairs[(word[i], word[i+1])] += 1
    return pairs

def merge_pair(tokens, pair):
    first, second = pair
    new_tokens = []
    for word in tokens:
        w = []
        i = 0
        while i < len(word):
            if i < len(word)-1 and word[i]==first and word[i+1]==second:
                w.append(first+second)
                i += 2
            else:
                w.append(word[i])
                i += 1
        new_tokens.append(w)
    return new_tokens

# -----------------------
# Step 1: Learn 30 merges
learned_merges = []
for step in range(30):
    pairs = get_stats(corpus)
    if not pairs:
        break
    top_pair = pairs.most_common(1)[0][0]
    corpus = merge_pair(corpus, top_pair)
    learned_merges.append(top_pair)

# Flatten vocabulary
vocab = set(sym for word in corpus for sym in word)

# -----------------------
# Step 2: Five most frequent merges & 5 longest subword tokens
print("Top 5 most frequent merges:", learned_merges[:5])
longest_subwords = sorted(vocab, key=lambda x: len(x), reverse=True)[:5]
print("5 longest subword tokens:", longest_subwords)

# -----------------------
# Step 3: Segment 5 words from paragraph
def segment_word(word, merges):
    tokens = list(word) + ["_"]
    changed = True
    while changed:
        changed = False
        for pair in merges:
            i = 0
            new_tokens = []
            while i < len(tokens):
                if i < len(tokens)-1 and (tokens[i], tokens[i+1]) == pair:
                    new_tokens.append(tokens[i]+tokens[i+1])
                    i += 2
                    changed = True
                else:
                    new_tokens.append(tokens[i])
                    i += 1
            tokens = new_tokens
    return tokens

words_to_segment = ["transforming", "computers", "recognition", "datasets", "Innovative"]  # rare + derived
print("\nSubword segmentations:")
for w in words_to_segment:
    print(f"{w} --> {segment_word(w, learned_merges)}")

# -----------------------
# Step 4: Reflection
reflection = """
BPE learned subwords including stems (learn, perform, recogn), suffixes (-ing, -s, -ion), and some whole words (AI, but).
Prefixes were less frequent due to small corpus size.
Subword tokenization allows models to handle rare or unseen words like 'Innovative' or 'transforming' by splitting them into known subunits, reducing OOV issues.
It also captures meaningful morphemes, e.g., 'ing_' as verb suffix.
Pros: (1) Handles rare words and morphological variants. (2) Reduces vocabulary size compared to word-level tokens.
Cons: (1) Segmentation may split semantically important units awkwardly. (2) Context for some words may be harder to capture when split.
Overall, BPE balances vocabulary efficiency and OOV handling for English text.
"""
print("\nReflection:", reflection)


Top 5 most frequent merges: [('e', '_'), ('i', 'n'), ('a', 'r'), ('s', '_'), ('a', 'c')]
5 longest subword tokens: ['learning_', 'learning', 'achine_', 'learn', 'form']

Subword segmentations:
transforming --> ['t', 'r', 'an', 's', 'form', 'ing', '_']
computers --> ['co', 'mp', 'u', 't', 'er', 's_']
recognition --> ['re', 'co', 'g', 'n', 'it', 'i', 'o', 'n', '_']
datasets --> ['d', 'at', 'a', 'se', 't', 's_']
Innovative --> ['I', 'n', 'n', 'o', 'v', 'at', 'i', 've_']

Reflection: 
BPE learned subwords including stems (learn, perform, recogn), suffixes (-ing, -s, -ion), and some whole words (AI, but). 
Prefixes were less frequent due to small corpus size. 
Subword tokenization allows models to handle rare or unseen words like 'Innovative' or 'transforming' by splitting them into known subunits, reducing OOV issues. 
It also captures meaningful morphemes, e.g., 'ing_' as verb suffix. 
Pros: (1) Handles rare words and morphological variants. (2) Reduces vocabulary size compared to word-le